In [10]:
import uuid
import json
import copy
from pathlib import Path

class NeodashEditor:
    def __init__(self, json_data):
        if isinstance(json_data, (str, Path)):
            path = Path(json_data)
            if path.exists():
                with open(path, 'r') as f:
                    self.data = json.load(f)
            else:
                self.data = json.loads(json_data)
        elif isinstance(json_data, dict):
            self.data = json_data
        else:
            raise TypeError("Input must be a JSON string, a Path, or a dictionary.")

        self.refresh()

    def refresh(self):
        self.existing_ids = self._collect_all_report_ids()
        self.report_index = self._collect_title_id_page_pairs()
        self.page_titles = self._collect_page_titles()

    def _collect_all_report_ids(self):
        ids = set()
        for page in self.data.get("pages", []):
            for report in page.get("reports", []):
                ids.add(report.get("id"))
        return ids

    def _collect_title_id_page_pairs(self):
        index = []
        for page_index, page in enumerate(self.data.get("pages", [])):
            for report in page.get("reports", []):
                title = report.get("title")
                rid = report.get("id")
                if title and rid:
                    index.append((title, rid, page_index))
        return index

    def _collect_page_titles(self):
        return [page.get("title", f"Page {i}") for i, page in enumerate(self.data.get("pages", []))]

    def _generate_unique_id(self):
        new_id = str(uuid.uuid4())
        while new_id in self.existing_ids:
            new_id = str(uuid.uuid4())
        self.existing_ids.add(new_id)
        return new_id

    def find_report_by_title(self, title):
        for i, page in enumerate(self.data.get("pages", [])):
            for j, report in enumerate(page.get("reports", [])):
                if report.get("title") == title:
                    return i, j, report
        return None, None, None

    def duplicate_report(self, title, new_title=None, target_page_index=0, new_position=(0, 0)):
        page_index, report_index, original = self.find_report_by_title(title)
        if original is None:
            raise ValueError(f"Report with title '{title}' not found.")

        duplicate = copy.deepcopy(original)
        duplicate['id'] = self._generate_unique_id()
        if new_title:
            duplicate['title'] = new_title
        duplicate['x'], duplicate['y'] = new_position

        self.data['pages'][target_page_index]['reports'].append(duplicate)
        return duplicate

    def add_report_to_page(self, report, page_index):
        report_copy = copy.deepcopy(report)
        report_copy['id'] = self._generate_unique_id()
        self.data['pages'][page_index]['reports'].append(report_copy)
        return report_copy

    def duplicate_page(self, page_index, new_title=None):
        if page_index >= len(self.data['pages']):
            raise IndexError("Page index out of range.")

        original_page = self.data['pages'][page_index]
        new_page = copy.deepcopy(original_page)
        new_page_title = new_title or f"{original_page['title']} (Copy)"
        new_page['title'] = new_page_title

        for report in new_page.get("reports", []):
            report['id'] = self._generate_unique_id()

        self.data['pages'].append(new_page)
        self.page_titles.append(new_page_title)
        return new_page

    def remove_reports_by_uid(self, uids):
        if isinstance(uids, str):
            uids = [uids]
        for page in self.data.get("pages", []):
            page['reports'] = [r for r in page.get('reports', []) if r.get('id') not in uids]
        self.existing_ids -= set(uids)
        self.report_index = self._collect_title_id_page_pairs()

    def remove_page_by_index(self, page_index):
        if 0 <= page_index < len(self.data['pages']):
            del self.data['pages'][page_index]
            self.page_titles = self._collect_page_titles()
            self.report_index = self._collect_title_id_page_pairs()
            self.existing_ids = self._collect_all_report_ids()
        else:
            raise IndexError("Page index out of range.")

    def to_json(self, new_uuid=True):
        output = copy.deepcopy(self.data)
        if new_uuid:
            output['uuid'] = self._generate_unique_id()
        return json.dumps(output, indent=2)

    def save_to_file(self, path, new_uuid=True):
        with open(path, 'w') as f:
            f.write(self.to_json(new_uuid=new_uuid))

    def list_all_report_titles(self):
        self.refresh()
        return self.report_index

    def list_all_page_titles(self):
        self.refresh()
        return self.page_titles


In [11]:
ne = NeodashEditor("/home/patch/Downloads/dashboard (6).json")
ne.list_all_report_titles()

[('Labs', '3a578498-3af1-4557-826f-bd998d505a6e', 0),
 ('Add Lab', 'a4fed60e-769b-4706-a1f5-50e0dfd90107', 0),
 ('Add/Edit Person (MATCH TO EDIT: Name and Email)',
  '04fb1b9a-e79c-45d9-8223-872652a10fbe',
  2),
 ('Staff', '875d95b4-5551-48d4-af1d-a54faf9d068f', 2),
 ('Remove Person', '405ea5a2-9b38-4ecf-a92c-f8db304732a7', 2),
 ('People', '9ad0ef27-fe28-4ee8-a5f6-da7d31b2b530', 2),
 ('Search Results', '095926b1-b3d3-4639-b370-e4a6cc619eb7', 2),
 ('Search by Name', 'fa530764-bde4-4a58-959a-2c8bf9001a65', 2),
 ('Users', '66263c25-5116-48d0-9d93-d29ff7f9e650', 2),
 ('Add Study', '12e08993-dcf1-40c7-86b2-0583f9d4220f', 5),
 ('Add Person', 'c9ccd5fe-c9d3-4854-b83e-49e53277c38c', 10),
 ('Staff', 'b7e453ff-246e-4015-80be-9e5eecd992ff', 10),
 ('Remove Person', 'b0c8514e-9d8b-418a-899c-fc98446e521e', 10),
 ('People', '15f6fbfa-d768-4869-91c6-10b5194bbf05', 10),
 ('Search Results', 'ccf4426c-35e0-4d3c-9f7f-8b82e90199e8', 10),
 ('Search by Name', 'b2578e4c-bab5-4d4a-9f38-36a5baf156f0', 10),
 ('U

In [12]:
ne.duplicate_page(2, new_title='Labs_NEW')

{'title': 'Labs_NEW',
 'reports': [{'id': '39b1634a-6a3a-466f-a110-2b6f0375dd55',
   'title': 'Add/Edit Person (MATCH TO EDIT: Name and Email)',
   'query': 'MERGE (p:Person {\n  name: $neodash_person_name\n})\nSET p.email = $neodash_person_email\nSET p.rolePMIT = $neodash_person_rolepmit\nSET p.affiliation = $neodash_lab_name\nRETURN p',
   'width': 8,
   'height': 6,
   'x': 12,
   'y': 0,
   'type': 'forms',
   'selection': {},
   'settings': {'formFields': [{'type': 'Free Text',
      'settings': {'type': 'Free Text',
       'entityType': 'person_name',
       'parameterName': 'neodash_person_name'},
      'query': 'RETURN true;'},
     {'type': 'Free Text',
      'settings': {'type': 'Free Text',
       'entityType': 'person_email',
       'parameterName': 'neodash_person_email'},
      'query': 'RETURN true;'},
     {'type': 'Node Property',
      'settings': {'type': 'Node Property',
       'entityType': 'Person',
       'propertyType': 'rolePMIT',
       'propertyTypeDisplay': 

In [13]:
ne.save_to_file('dashboard_update_test.json')